In [1]:
import os
import sys
sys.path.append(os.getcwd() + '/src')
from functions import getCodes


def find_node(line, keys, false_keys, comment_keys):
    for start_key in keys:
        if line.find(start_key) >= 0:
            start = line.find(start_key) + len(start_key)
            end = len(line) + 1
            for stop_code in keys[start_key]:
                if line[start:].find(stop_code) >= 0:
                    if (start > len(start_key) and ' ' in keys[start_key]) and line[start-len(start_key)-1] != ' ':
                        continue
                    end = min(start + line[start:].find(stop_code), end)
            if end > len(line):
                continue

            node = line[start:end]
            key = True

            for comment_key in comment_keys:
                if line.find(comment_key) >= 0 and line.find(comment_key) < line.find(start_key): 
                    key = False
                    break
            if not key: continue

            for false_key in false_keys:
                if node.find(false_key) == 0: 
                    key = False
                    break
            if not key: continue
            
            if node != '':
                if node[0] == '"' and node[-1] == '"':
                    node = node[1:-1]
                if node.find("$sourcetablesdb") >= 0:
                    node = 'derived' + node[node.find('.'):-1]
                return node
    return None

def parse(file):
    result = {'input': {}, 'output': {}, 'cte': {}}
    input_keys = {}
    input_keys['sql'] = {'from ': [' ', '"""', '\t', ')'], 'join ': [' ', '\t', ')'], 'using ': [' ', '\t', ')'], 'table_changes("': ['"']}
    input_keys['scala'] = {'read.table(': [')']}
    output_keys = {}
    output_keys['sql'] = {'merge ': [' ', '\t'], 'insert ': [' ', '\t'],'overwrite ': [' ', '\t'],'table ': [' ', '\t']}
    output_keys['scala'] = {'saveastable(': [')']}
    cte_keys = {'with ' : [' as'], ',' : [' as'], ', ': [' as']}
    comment_keys = {'//', '--'}
    false_keys = ['select', 'into', 'table', 'overwrite', '*', 'if', '(', 'table_changes']

    with open(codes[file]['path'], 'r') as f:
        text = ""
        filetype = codes[file]['language']
        for line in f:
            text = text + line
            line = line.lower()[:-1] + ' '
            input_key = find_node(line, input_keys[filetype], false_keys, comment_keys)
            if input_key is not None:
                result['input'][input_key] = (file, line)
            output_key = find_node(line, output_keys[filetype], false_keys, comment_keys)
            if output_key is not None:
                result['output'][output_key] = (file, line)
        
        cte = {}
        for input_key in result['input']:
            if text.find(f"{input_key} as") >= 0:       #CTE
                cte[input_key] = True
        for input_key in cte:
            result['input'].pop(input_key)

    return result

In [6]:
def getEdges():

    codes = getCodes(os.getcwd())
    nodes = {'input': {}, 'output': {}}
    edges = []
    #check = ['']
    for file in codes: 
        #if file.find('C125') < 0: continue
        result = parse(file)
        for type in result:
            for node in list(result[type].keys()):
                if node in check: print(f"[{node}]\n{result[type][node]}\n")
                nodes[type][node] = (nodes[type][node] + 1) if node in nodes[type] else 1
        for a in list(result['input'].keys()):
            for b in list(result['output'].keys()):
                edges.append((a, b))

    print(len(edges))
    return edges

"""
for type in nodes:
    print(type)
    for node in sorted(list(nodes[type].keys())):
        print(f"{node:40s}{nodes[type][node]}")
"""

1348


'\nfor type in nodes:\n    print(type)\n    for node in sorted(list(nodes[type].keys())):\n        print(f"{node:40s}{nodes[type][node]}")\n'

In [30]:
import sys
sys.path.append(os.getcwd() + '/src')
from functions import getCodes
codes = getCodes(os.getcwd())
nodes = {'input': {}, 'output': {}}
edges = []
check = ['']
omop_table = ['person', 'observation_period', 'visit_occurrence', 'visit_detail'
'condition_occurrence',
'drug_exposure',
'procedure_occurrence'
,'device_exposure'
,'measurement'
,'observation'
,'death'
,'location'
,'care_site'
,'provider'
,'payer_plan_period'
,'cost'
,'drug_era'
,'dose_era'
,'condition_era'
,'episode'
,'episode_event'
,'metadata'
,'cdm_source'
,'concept'
,'vocabulary'
]
for file in codes: 
    #if file.find('C125') < 0: continue
    result = parse(file)
    for type in result:
        for node in list(result[type].keys()):
            if node in check: print(f"[{node}]\n{result[type][node]}\n")
            nodes[type][node] = (nodes[type][node] + 1) if node in nodes[type] else 1
    for a in list(result['input'].keys()):
        for b in list(result['output'].keys()):
            #if b in omop_table:
            edges.append((a, b))

print(len(edges))

"""
for type in nodes:
    print(type)
    for node in sorted(list(nodes[type].keys())):
        print(f"{node:40s}{nodes[type][node]}")
"""

1348


'\nfor type in nodes:\n    print(type)\n    for node in sorted(list(nodes[type].keys())):\n        print(f"{node:40s}{nodes[type][node]}")\n'

In [31]:
import streamlit as st
import networkx as nx
from pyvis.network import Network
from streamlit.components.v1 import html

def create_dag(edges):
    G = nx.DiGraph()
    G.add_edges_from(edges)
    print(nx.is_directed_acyclic_graph(G))
    return G

def draw_graph(G, highlight=None):
    nt = Network('500px', '500px', directed=True, notebook=False)
    nt.from_nx(G)

    if highlight:
        # Highlight the selected node and its neighbors
        for node in nt.nodes:
            if node['id'] == highlight or node['id'] in [n for n in nx.all_neighbors(G, highlight)]:
                node['color'] = 'red'
            else:
                node['color'] = '#D2E5FF'
            print('exec')
    
    # Save the network to an HTML file
    nt.save_graph('dag.html')



In [32]:
def app():
    G = create_dag(edges)
    
    #st.title('Interactive DAG Visualization')
    #selected_node = st.selectbox('Select a node to highlight', options=G.nodes())
    
    # Draw graph with or without a highlight and save to HTML
    draw_graph(G)

    # Display the HTML file in Streamlit
    #HtmlFile = open('dag.html', 'r', encoding='utf-8')
    #source_code = HtmlFile.read() 
    #html(source_code, height=500)

app()


False


In [ ]:
for a, b in edges:
    if a.find('clarity.')>=0: print(a)

In [40]:
import streamlit as st
import networkx as nx
import matplotlib.pyplot as plt

# Create a sample directed graph
G = nx.DiGraph()
G.add_edge('A', 'B')
G.add_edge('B', 'C')
G.add_edge('C', 'A')

# Draw the graph
plt.figure(figsize=(8, 8))
nx.draw(G, with_labels=True, node_color='skyblue', edge_color='k', node_size=2000, arrowstyle='-|>', arrowsize=20)
st.pyplot(plt)


DeltaGenerator()